# 010. Regular Expression Matching 正则表达式匹配

### 难度：Hard

## 刷题内容

> 原题链接

 - 中文：https://leetcode-cn.com/problems/two-sum/description
 - 英文：https://leetcode.com/problems/two-sum

> 内容描述

```
给定一个字符串 (s) 和一个字符模式 (p)。实现支持 '.' 和 '*' 的正则表达式匹配。

'.' 匹配任意单个字符。
'*' 匹配零个或多个前面的元素。
匹配应该覆盖整个字符串 (s) ，而不是部分字符串。

说明:
 - s 可能为空，且只包含从 a-z 的小写字母。
 - p 可能为空，且只包含从 a-z 的小写字母，以及字符 . 和 *。

示例 1:
输入:
s = "aa"
p = "a"
输出: false
解释: "a" 无法匹配 "aa" 整个字符串。

示例 2:
输入:
s = "aa"
p = "a*"
输出: true
解释: '*' 代表可匹配零个或多个前面的元素, 即可以匹配 'a' 。因此, 重复 'a' 一次, 字符串可变为 "aa"。

示例 3:
输入:
s = "ab"
p = ".*"
输出: true
解释: ".*" 表示可匹配零个或多个('*')任意字符('.')。

示例 4:
输入:
s = "aab"
p = "c*a*b"
输出: true
解释: 'c' 可以不被重复, 'a' 可以被重复一次。因此可以匹配字符串 "aab"。

示例 5:
输入:
s = "mississippi"
p = "mis*is*p*."
输出: false
```

## 解题方案

> 思路 1

暴力法。Lisanaaa 大佬想出来的，不得不赞一个，哈哈哈。之前想暴力法，把脑袋想爆掉了都没想全各种情况。那么，看看他是怎么解的吧。

"." 很容易处理。难点就在 "*" 身上， "*" 是不会单独的，它一定是和前面的 一个字母或者 "." 配成一对 出现的。看成一对后是这样子的 "X*" ，它的性质是：要么匹配 0 个，要么匹配连续的 "X" 。所以尝试暴力法的时候一个 trick 是从后往前匹配。

暴力解法也是可以 AC 的。

下面这样来分情况：

 - 如果 s[i] = p[j] 或者 p[j]= . ： 往前匹配一位
 - 如果 p[j] = ' * ' , 检查一下，如果这个时候 p[j-1] = . 或者 p[j-1] = s[i] ，那么就往前匹配，如果这样能匹配过，就 return True ， 否者我们忽略 'X* ' ,这里注意里面的递推关系。
 - 再处理一下边界情况：
  - s 已经匹配完了， 如果此时 p 还有，那么如果剩下的是 X* 这种可以过，所以需要检查一下
  - p 匹配完毕，如果 s 还有没有匹配的话，那么就 return False

In [1]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        def helper(s, i, p, j):
            if j == -1:
                return i == -1
            if i == -1:
                if p[j] != '*':
                    return False
                return helper(s, i, p, j-2)
            if p[j] == '*':
                if p[j-1] == '.' or p[j-1] == s[i]:
                    if helper(s, i-1, p, j):
                        return True
                return helper(s, i, p, j-2)
            if p[j] == '.' or p[j] == s[i]:
                return helper(s, i-1, p, j-1)
            return False

        return helper(s, len(s)-1, p, len(p)-1)
    
s = 'abc'
p = 'a*abc'
ss = Solution()
print(ss.isMatch(s, p))

True


> 思路 2

动态规划 解法。

dp 优化，感觉和 edit distance 很像。DP优化待代码化，感觉学DP的一个重点除了递归学好以外，另一点是一定要会画表格。

画一个表格来看一下状况

```
			c	*	a	*	b
		0	1	2	3	4	5
	0	1	0	1	0	1	0		
a	1	0	0	0	1	1	0					
a	2	0	0	0	0	1	0					
b	3	0	0	0	0	0	1			
```

这里有几个取巧/容易出问题的敌方，这里画的表用的是1-based string。一上来，做的事包括：

 - 初始化，空字符匹配：dp[0][0] =1
 - 第一行， c * 可以匹配空字符， c * a * 可以匹配空字符， p[j-1] != s[i] ，匹配空字符
 - 然后进入第二行再来看，实际上我们可以看到，如果没有碰到 * 匹配还是很朴素的，但是碰到 * ：
  - 1 这个匹配可以从左侧传来，dp[i][j] = dp[i][j-1] ，that is 匹配 1 个
  - 1 也可以有上方传来，这种情况是 p[j-1] = s[i] ，匹配多个 dp[i][j] = dp[i-1][j]
  - 1 这个匹配也可以从间隔一个的左侧传来，that is 也可以有个性的匹配 0 个,如同匹配空字符一样 dp[i][j] = dp[i][j-2] ，但是注意匹配 0 个实际上有两种状况，如果 p[j-1]!=s[i] ,强制匹配 0 个，即使 p[j-1] == s[i] ,我们也可以傲娇的用它来匹配 0 个。
  
再代码化一点：

 - s[i] == p[j] 或者 p[j] == '.' : dp[i][j] = dp[i-1][j-1]
 - p[j] == '*' : 然后分几种情况
  - p[j-1] != s[i] : dp[i][j] = dp[i][j-2] 匹配 0 个的状况
  - p[j-1] == s[i] or p[i-1] == '.':
    - dp[i][j] = dp[i-1][j] 匹配多个 s[i]
    - dp[i][j] = dp[i][j-2] 匹配 0 个

AC 代码，注意一下，因为上表为了表达方便，用的是 1-based string 系统，实际写代码的时候我们心里还是清楚这个 string 还是从 0 开始的，不过也可以尝试往前面添东西来方便。

In [2]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        m, n = len(s), len(p)
        dp = [ [0 for i in range(n+1)] for j in range(m+1)]

        dp[0][0] = 1

        # init the first line
        for i in range(2,n+1):
            if p[i-1] == '*':
                dp[0][i] = dp[0][i-2]

        for i in range(1,m+1):
            for j in range(1,n+1):
                if p[j-1] == '*':
                    if p[j-2] != s[i-1] and p[j-2] != '.':
                        dp[i][j] = dp[i][j-2]
                    elif p[j-2] == s[i-1] or p[j-2] == '.':
                        dp[i][j] = dp[i-1][j] or dp[i][j-2]

                elif s[i-1] == p[j-1] or p[j-1] == '.':
                    dp[i][j] = dp[i-1][j-1]

        return dp[m][n] == 1 
    
s = 'abc'
p = 'a*abc'
ss = Solution()
print(ss.isMatch(s, p))

True


我做的动态规划和上面大体思路是类似的。

直接说动态转移方程组了，我们首先定义一些变量方便解释，被匹配串为s，模式串为p。状态转移数组 dp[i][j]表示利用 p 的前 j 个字符匹配 s 的前 i 个字符的匹配结果（成功为 true，失败为 false）。

 - **s[i] == p[j] || p[j] == '.'** ，那么 dp[i][j] = dp[i-1][j-1] ，也就是既然 s 串的第 i 个字符能和 p 串的第 j 个字符匹配，那么如果 s 串的前 i-1 个字符和 p 串的前 j-1 个字符能匹配则 s 串的前 i 个和 p 串的前 j 个则能匹配，反之不能。
 - **p[j] == '*'** ：分情况讨论。
   - **s[i] != p[j-1] && p[j-1] != '.'** ，那么 dp[i][j] = dp[i][j-2] ，也就是 * 号前面的那个字符在匹配的过程当中一个都不使用。
   - **else，那么dp[i][j] = dp[i-1][j] || dp[i][j-1] || dp[i][j-2]**，也就是说要么使用 '*' 号进行匹配( dp[i-1][j] )，要么只使用 '*' 号前面的那个字符匹配，不使用 '*' 匹配( dp[i][j-1] )，要么 '*' 号前面的那个字符在匹配的过程当中一个都不使用（ dp[i][j-2] ），只要有一个是 true 则能够匹配。
   
最后考虑一下边界条件。一开始 i 是从 1 到 len(s) ，j 是 1 到 len(p) 。首先一来就能想到 dp[0][0] 是true，其他都是 false。但是这个边界条件是不够的。比如 isMatch("aab", "c * a * b") ，dp[1][3] 应该是从 dp[0][2] 转移过来的，所以需要更多的边界条件，也就是一开始的 * 是能匹配空字符串的。所以我把 i 改到了从 0 开始,并且第二条也添加了 i=0，dp[i][j] = dp[i][j-2] 。

In [3]:
class Solution:
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        dp = []
        dp = [[False for i in range(len(p)+1)] for j in range(len(s)+1)]
        dp[0][0] = True
#         print(dp)
        for i in range(len(s)+1):
            for j in range(1, len(p)+1):
                if i > 0 and (s[i-1] == p[j-1] or p[j-1] == '.'):
#                     print("1111111",i)
#                     print("2222222",j)
                    dp[i][j] = dp[i-1][j-1]
#                     print("3333333",dp[i-1][j-1])
                if p[j-1] == '*':
                    if i == 0 or (s[i-1] != p[j-2] and p[j-2] != '.'):
#                         print("4444444",i)
#                         print("5555555",j)
                        dp[i][j] = dp[i][j-2]
#                         print("6666666",dp[i][j-2])
                    else:
                        dp[i][j] = dp[i-1][j] or dp[i][j-1] or dp[i][j-2]
#                         print("7777777",i)
#                         print("8888888",j)
#         print(dp)                       
        return dp[len(s)][len(p)]
    
s = 'aa'
p = 'a*'
ss = Solution()
print(ss.isMatch(s, p))

True
